In [1]:
#default_exp httpadapter

# HTTP Adapter 

> HTTP Adapter for chatbot using flask

In [ ]:
# export
from flask import request, Flask
import json
import redis

"""
HTTP based adapter to invoke root element on user reponse.
"""
app = Flask(__name__)
STATE = {}

def init(root):
    """
    Pass root node of chatbot and start server. 
    This function does not return till server is running
    """
    STATE['root'] = root
    STATE['r'] = redis.Redis()
    STATE['r'].flushall()
    app.run(port=5000, debug=True)

def get_data(session):
    """
    Get data for session
    """
    if STATE['r'].exists(session):
        data = json.loads(STATE['r'].get(session))
    else:
        data = {}
    return data

@app.route('/chat', methods=['POST'])
def respond():
    """
    Respond method bound to chat REST endpoint
    """
    print(f'{request.form}')
    inp = request.form['user_input']
    session = request.form['session']
    data = get_data(session)
    n, d1 = STATE['root'].answer(inp, data)
    d2 = STATE['root'].question(data)
    # debug_print(f'adapter data {data}')
    STATE['r'].set(session, json.dumps(data))
    # return f"{d1['txt']}\n{d2['txt']}"
    out = {
            'text': '<br>'.join([d1['txt'], d2['txt']])
          }
    
    if 'choices' in d2:
        out['b_flag'] = 1
        out['btns'] = [{'value': i+1, 'text': x} for (i, x) in 
                       enumerate(d2['choices'])]
    print(data)
    return json.dumps(out)

In [1]:
# hide
#             {'value': 'a1', 'text': 'First Button'}, 
#                 {'value': 'a2', 'text': 'Second Button'},
#                 {'value': 'a3', 'text': 'Third Button with really long text to test multiline'+
#                     ' Second line added here'}, {'value': 'a4', 
#                         'text': 'Fourth Button with really long text to test multiline'+
#                                             ' Second line added here'}],
    # btns; key for buttons
    # value, text: fields
    

## Sample test code

In [ ]:
import flowchatbot.httpadapter as http
from flowchatbot.flowchatbot import Composite, Segment, MultiChoiceSegment

bot = Composite('key1',
                  Segment('key2', '', 'Welcome to QnA bot'),
                  Segment('key3', 'Your name?', 'got it'),
                  Segment('key4', 'Your phone?', 'got it'),
                  Segment('key5', 'Your email?', 'got it'),
                  MultiChoiceSegment('key6', 'Your income bracket?', 
                      [' < 1L', '1L - 10L', '> 10L'], 'got it')
                           )

http.init(bot)